# Weather AI Model

**Importing Libraries:**

In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [2]:
API_KEY = '0fcf800b3c02d66198fed2754bd6afbe'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

**fetch current weather data**

In [3]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'temp_min': round(data['main']['temp_min']),
      'temp_max': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'wind_gust_speed': data['wind']['speed']
  }

**Read historical data**

In [11]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

**Prepare data for training**

In [15]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  X = data[['MinTemp', 'MaxTemp', 'WindGustDir', 'WindGustSpeed', 'Humidity', 'Pressure', 'Temp']] # feature variables
  Y = data['RainTomorrow'] # target variable

  return X, Y, le

**Train Rain Prediction Model**

In [17]:
def train_rain_model(X, Y):
  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train, y_train) # train

  y_pred = model.predict(x_test)

  # test
  print("Mean Squared Error for Rain Model")
  print(mean_squared_error(y_test, y_pred))

  return model


**Prepare regression data**

In [21]:
def prepare_regression_data(data, feature):
  X, Y = [], []

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    Y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1, 1)
  Y = np.array(Y)
  return X, Y

**Train regression Model**

In [8]:
def train_regression_model(X, Y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, Y)
  return model

**Predict**

In [9]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])

  return predictions[1:]

**Weather Analysis Function**

In [26]:
def weather_view():
  city = input('Enter any city name: ')
  current_weather = get_current_weather(city)

  historical_data = read_historical_data('/content/weather.csv')

  x, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(x, y)

  wind_deg = current_weather['wind_gust_dir'] %360
  compass_points = [
      ("N", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
      ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
      ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
      ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
      ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
      ("NNW", 326.25, 348.75)
    ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir':compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],
  }
  current_df = pd.DataFrame([current_data])

  rain_prediction = rain_model.predict(current_df)[0] # rain prediction

  x_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  x_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(x_temp, y_temp)
  hum_model = train_regression_model(x_hum, y_hum)

  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity =  predict_future(hum_model, current_weather['humidity'])

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  # Display results
  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temprature: {current_weather['current_temp']}")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Minimum Temprature: {current_weather['temp_min']}˚C")
  print(f"Maximum Temprature: {current_weather['feels_like']}˚C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temprature Prediction:")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}˚C")

  print("\nFuture Humidity Prediction:")
  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")

weather_view()

Enter any city name: xian
Mean Squared Error for Rain Model
0.1506849315068493
City: xian, CN
Current Temprature: 1
Feels Like: -2
Minimum Temprature: 1˚C
Maximum Temprature: -2˚C
Humidity: 77%
Weather Prediction: overcast clouds
Rain Prediction: No

Future Temprature Prediction:
21:00: 8.9˚C
22:00: 11.2˚C
23:00: 11.7˚C
00:00: 11.6˚C
01:00: 11.7˚C

Future Humidity Prediction:
21:00: 61.9%
22:00: 56.5%
23:00: 40.7%
00:00: 52.9%
01:00: 51.0%
